<a target="_blank" href="https://colab.research.google.com/github/giordamaug/HELP/blob/main/HELPpy/notebooks/experiment.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://www.kaggle.com/notebooks/welcome?src=https://github.com/giordamaug/HELP/blob/main/HELPpy/notebooks/experiment.ipynb">
  <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/>
</a>

### 1. Install HELP from GitHub
Skip this cell if you already have installed HELP.

In [ ]:
!pip install git+https://github.com/giordamaug/HELP.git

### 2. Download the input files
For a chosen tissue (here `Kidney`), download from GitHub the label file (here `Kidney_HELP.csv`, computed as in Example 1) and the attribute files (here BIO `Kidney_BIO.csv`, CCcfs `Kidney_CCcfs_1.csv`, ..., `Kidney_CCcfs_5.csv`, and N2V `Kidney_EmbN2V_128.csv`). 
Skip this step if you already have these input files locally.

In [ ]:
tissue='Kidney'
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_HELP.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_BIO.csv
for i in range(5):
  !wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_CCcfs_{i}.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_EmbN2V_128.csv
#!wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_CCBeder.csv

Other attribute files (CCBeder) are shown but commented to help the user experiment with different data.

### 3. Download the script for the experiments and show the man page
Download the batch script for EG prediction used for the experiments and show its manual page:

In [1]:
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/HELPpy/notebooks/EG_prediction.py
!python EG_prediction.py -h

usage: EG_prediction.py [-h] -i <inputfile> [<inputfile> ...]
                        [-c <chunks> [<chunks> ...]]
                        [-X <excludelabels> [<excludelabels> ...]]
                        [-L <labelname>] -l <labelfile> [-A <aliases>]
                        [-b <seed>] [-r <repeat>] [-f <folds>] [-j <jobs>]
                        [-B] [-v <voters>] [-ba] [-fx] [-n <normalize>]
                        [-o <outfile>] [-s <scorefile>] [-p <predfile>]

PLOS COMPBIO

options:
  -h, --help            show this help message and exit
  -i <inputfile> [<inputfile> ...], --inputfile <inputfile> [<inputfile> ...]
                        input attribute filename list
  -c <chunks> [<chunks> ...], --chunks <chunks> [<chunks> ...]
                        no of chunks for attribute filename list
  -X <excludelabels> [<excludelabels> ...], --excludelabels <excludelabels> [<excludelabels> ...]
                        labels to exclude (default NaN, values any list)
  -L <labelname>,

### 4. Run the E vs NE experiments
This cell's code reproduces the results for Kidney reported in Table 3 (A) of the HELP paper. 

In [2]:
tissue = "Kidney"                               # or 'Lung'
labelfile = f"{tissue}_HELP.csv"                # label filename
aliases = "-A \"{'aE': 'NE', 'sNE':'NE'}\""     # dictionary for renaming labels before prediction: es. {'oldlabel': 'newlabel'}
excludeflags = ""                               # label to remove (none for E vs NE problem)
njobs = "-1"                                    # parallelism level: -1 = all cpus, 1 = sequential
nchunks = "-c 1 5 1"                            # no. of chunks for each input attribute file: es. 1 5 (Bio is one chunk, CCcfs split in 5 chunks)
voters = "-v 3"                                 # no. of voters on classifier ensemble
repeats = "-r 10"                               # no. of iterations for experiments 
!python EG_prediction.py -i {tissue}_BIO.csv \
                            {tissue}_CCcfs.csv \
                            {tissue}_EmbN2V_128.csv \
                            {nchunks} \
                            -l {datapath}/{labelfile} \
                            {aliases} {excludeflags}  \
                            {voters} {repeats} \
                            -n std -ba \
                            -j {njobs} -B

- replacing label aE with NE
- replacing label sNE with NE
- removing label []
Majority NE 16678 minority E 1253
[Kidney_BIO.csv] found 52532 Nan...
[Kidney_BIO.csv] Normalization with std ...
Loading file in chunks: 100%|█████████████████████| 5/5 [00:02<00:00,  2.03it/s]
[Kidney_CCcfs.csv] found 6676644 Nan...
[Kidney_CCcfs.csv] Normalization with std ...
[Kidney_EmbN2V_128.csv] found 0 Nan...
[Kidney_EmbN2V_128.csv] No normalization...
17236 labeled genes over a total of 17931
(17236, 3456) data input
Running par on 8 cpus...
{'E': 0, 'NE': 1}
label
NE       15994
E         1242
Name: count, dtype: int64{'E': 0, 'NE': 1}
label
NE       15994
E         1242
Name: count, dtype: int64
{'E': 0, 'NE': 1}
label
NE       15994
E         1242
Name: count, dtype: int64
{'E': 0, 'NE': 1}
label
NE       15994
E         1242
Name: count, dtype: int64

{'E': 0, 'NE': 1}
label
NE       15994
E         1242
Name: count, dtype: int64
{'E': 0, 'NE': 1}
label
NE       15994
E         1242
Name: count

### 5. Run the E vs sNE experiments
This cell's code reproduces the results for Kidney reported in Table 4 (A) of the HELP paper, removing the `aE` flags (`excludeflags = "-X aE"`). 

In [3]:
tissue = "Kidney"                               # or 'Lung'
labelfile = f"{tissue}_HELP.csv"                # label filename
aliases = ""                                    # dictionary for renaming labels before prediction: es. {'oldlabel': 'newlabel'}
excludeflags = "-X aE"                          # label to remove: es. -X aE (for E vs sNE problem)
njobs = "-1"                                    # parallelism level: -1 = all cpus, 1 = sequential
nchunks = "-c 1 5 1"                            # no. of chunks for each input attribute file: es. 1 5 (Bio is one chunk, CCcfs split in 5 chunks)
voters = "-v 3"                                 # no. of voters on classifier ensemble
repeats = "-r 10"                               # no. of iterations for experiments 
!python EG_prediction.py -i {tissue}_BIO.csv \
                            {tissue}_CCcfs.csv \
                            {tissue}_EmbN2V_128.csv \
                            {nchunks} \
                            -l {datapath}/{labelfile} \
                            {aliases} {excludeflags}  \
                            -n std -ba \ 
                            {voters} {repeats} \
                            -j {njobs} -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs sNE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv Kidney_EmbN2V_128.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, sNE: 4810
SUBSAMPLE: 1:4
+-------------+--------------------------------+
|             | measure                        |
|-------------+--------------------------------|
| ROC-AUC     | 0.9701±0.0062                  |
| Accuracy    | 0.9354±0.0070                  |
| BA          | 0.9020±0.0120                  |
| Sensitivity | 0.8454±0.0231                  |
| Specificity | 0.9587±0.0065                  |
| MCC         | 0.8026±0.0215                  |
| CM          | [[10500, 1920], [1988, 46112]] |
+-------------+--------------------------------+


Please be aware that this will take a while in sequential execution. 